# Comparing channels

## Nucleotides
- R: sequential
- G: complementary
- B: not match at all

## Proteins
- sequential by BLOSUM62

In [236]:
import ppscore
import pandas
import os
import numpy
from scipy.stats import spearmanr
from IPython.core.display import display, HTML
from ete3 import Tree
from ete3.parser.newick import NewickError

/tmp/ipykernel_125167/1860748529.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [237]:
tree_path = "../../data/trees"
channels = ("N/full", "N/gray_r", "N/gray_g", "N/gray_b", "P/full")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin", "indelible"]

In [238]:
def do_euclidean(control, compare):
    return round(numpy.sqrt(numpy.sum((control.values - compare.values)**2)), 4)

def do_spearman(control, compare):
    return round(spearmanr(control.values.flatten(), compare.values.flatten())[0], 4)

def do_ppscore(control, compare):
    return round(ppscore.score(
                    pandas.DataFrame({"x": control.values.flatten(), "y": compare.values.flatten()}), "x", "y")["ppscore"], 4)

def do_rf(control, compare):
    return round(control.compare(compare, unrooted=True)["norm_rf"], 4)

def do_branch_score(control, compare):
    return 1.0 - round(control.compare(compare, unrooted=True)["source_edges_in_ref"], 4)


def read_and_compare(dataset, channel, score_func, control_name):
    result_dict = dict(
        algorithm=[],
        score=[],
        channel=[],
        dataset=[]
    )
    path = f"{tree_path}/{channel}/{dataset}"
    ends = control_name.split(".")[-1]
    if ends == "nw":
        control = Tree(f"{path}/{control_name}", format=1)
    else:
        control = pandas.read_csv(f"{path}/{control_name}", index_col=0)
    for file in os.listdir(path):
        if file.endswith(ends):
            basename = ".".join(file.split(".")[0:-1])
            try:
                if ends == "nw":
                    compare = Tree(f"{path}/{file}", format=1)
                else:
                    compare = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict["algorithm"] += [basename]
                result_dict["score"] += [score_func(control, compare)]
                result_dict["channel"] += [channel]
                result_dict["dataset"] += [dataset]
            except (FileNotFoundError, NewickError) as e:
                print(f"Error with file {path}/{file}")
    return result_dict

def pprintdf(score_func, control_name="Control with Clustal Omega.csv"):
    result_dict = dict(
        algorithm=[],
        score=[],
        channel=[],
        dataset=[]
    )
    for channel in channels:
        for dataset in datasets:
            d = read_and_compare(dataset, channel, score_func, control_name)
            result_dict["algorithm"] += d["algorithm"]
            result_dict["score"] += d["score"]        
            result_dict["channel"] += d["channel"]        
            result_dict["dataset"] += d["dataset"]
    df = pandas.DataFrame(result_dict)
    table = df.pivot(index=["algorithm", "channel"], columns="dataset", values="score")
    return table.head(50)

## Euclidean Distance from distance matrices

In [240]:
pprintdf(do_euclidean)

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       0.0000   
                                                   N/gray_b     0.0000   
                                                   N/gray_g     0.0000   
                                                   N/gray_r     0.0000   
                                                   P/full       0.0000   
Deep Search with Annoy                             N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       7.9242   
Global with Needleman-Wunsch                       N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       5.4879   
Greedy Sliced Structural Similarity Index Measure  N/full          NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   
Local with Smith–Waterman                          N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       4.3137   
MultiScale Structural Similarity Index Measure     N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full      36.9101   
Resized MultiScale Structural Similarity Index ... N/full          NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   
Resized Structural Similarity Index Measure        N/full          NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   
Structural Similarity Index Measure                N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full      26.1976   
Universal Quality Index                            N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full      22.4514   
Unrestricted Sliced Structural Similarity Index... N/full          NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   
Windowed MultiScale Structural Similarity Index... N/full          NaN   
                                                   N/gray_r        NaN   

dataset                                                      orthologs_androglobin  \
algorithm                                          channel       

## Linear Correlation from distance matrices

In [241]:
pprintdf(do_spearman)

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       1.0000   
                                                   N/gray_b     1.0000   
                                                   N/gray_g     1.0000   
                                                   N/gray_r     1.0000   
                                                   P/full       1.0000   
Deep Search with Annoy                             N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1590   
Global with Needleman-Wunsch                       N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.3039   
Greedy Sliced Structural Similarity Index Measure  N/full          NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   
Local with Smith–Waterman                          N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.2445   
MultiScale Structural Similarity Index Measure     N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1538   
Resized MultiScale Structural Similarity Index ... N/full          NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   
Resized Structural Similarity Index Measure        N/full          NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   
Structural Similarity Index Measure                N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.2689   
Universal Quality Index                            N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.2680   
Unrestricted Sliced Structural Similarity Index... N/full          NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   
Windowed MultiScale Structural Similarity Index... N/full          NaN   
                                                   N/gray_r        NaN   

dataset                                                      orthologs_androglobin  \
algorithm                                          channel       

## Predictive Power Score from distance matrices

In [242]:
pprintdf(do_ppscore)

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       0.9998   
                                                   N/gray_b     0.9998   
                                                   N/gray_g     0.9998   
                                                   N/gray_r     0.9998   
                                                   P/full       0.9992   
Deep Search with Annoy                             N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.4526   
Global with Needleman-Wunsch                       N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.8767   
Greedy Sliced Structural Similarity Index Measure  N/full          NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   
Local with Smith–Waterman                          N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.9643   
MultiScale Structural Similarity Index Measure     N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.2824   
Resized MultiScale Structural Similarity Index ... N/full          NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   
Resized Structural Similarity Index Measure        N/full          NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   
Structural Similarity Index Measure                N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.7029   
Universal Quality Index                            N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.7538   
Unrestricted Sliced Structural Similarity Index... N/full          NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   
Windowed MultiScale Structural Similarity Index... N/full          NaN   
                                                   N/gray_r        NaN   

dataset                                                      orthologs_androglobin  \
algorithm                                          channel       

## Robinson-foulds distance from generated Trees

In [243]:
pprintdf(do_rf, control_name="Control with Clustal Omega.nw")

dataset                                                      indelible  \
algorithm                                          channel               
Control with Clustal Omega                         N/full       0.0000   
                                                   N/gray_b     0.0000   
                                                   N/gray_g     0.0000   
                                                   N/gray_r     0.0000   
                                                   P/full       0.0000   
Deep Search with Annoy                             N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       1.0000   
Global with Needleman-Wunsch                       N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1892   
Greedy Sliced Structural Similarity Index Measure  N/full          NaN   
                                                   N/gray_r        NaN   
Local with Smith–Waterman                          N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1892   
MultiScale Structural Similarity Index Measure     N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.5135   
Resized MultiScale Structural Similarity Index ... N/full          NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   
Resized Structural Similarity Index Measure        N/full          NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   
Structural Similarity Index Measure                N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1892   
Universal Quality Index                            N/full          NaN   
                                                   N/gray_b        NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full       0.1892   
Unrestricted Sliced Structural Similarity Index... N/full          NaN   
                                                   N/gray_g        NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   
Windowed MultiScale Structural Similarity Index... N/full          NaN   
                                                   N/gray_r        NaN   
                                                   P/full          NaN   

dataset                                                      orthologs_androglobin  \
algorithm                                          channel       

In [213]:
pprintdf(do_rf, control_name="Global with Needleman-Wunsch.nw")

NewickError: Unexisting tree file or Malformed newick tree structure.
You may want to check other newick loading flags like 'format' or 'quoted_node_names'.

In [182]:
pprintdf(do_rf, control_name="Local with Smith–Waterman.nw")

NewickError: Unexisting tree file or Malformed newick tree structure.
You may want to check other newick loading flags like 'format' or 'quoted_node_names'.

## Compatibility branch score from generated Trees

In [172]:
pprintdf(do_branch_score, control_name="Control with Clustal Omega.nw")

dataset                                                     indelible  \
algorithm                                         channel               
Control with Clustal Omega                        N/full       0.0000   
                                                  N/gray_b     0.0000   
                                                  N/gray_g     0.0000   
                                                  N/gray_r     0.0000   
Greedy Sliced Structural Similarity Index Measure N/full          NaN   
                                                  N/gray_r        NaN   
                                                  P/full          NaN   
Universal Quality Index                           N/full          NaN   
                                                  N/gray_b        NaN   
                                                  N/gray_g        NaN   
                                                  N/gray_r        NaN   
                                                  P/full       0.0921   

dataset                                                     orthologs_androglobin  \
algorithm                                         channel                           
Control with Clustal Omega                        N/full                   0.0000   
                                                  N/gray_b                 0.0000   
                                                  N/gray_g                 0.0000   
                                                  N/gray_r                 0.0000   
Greedy Sliced Structural Similarity Index Measure N/full                      NaN   
                                                  N/gray_r                    NaN   
                                                  P/full                      NaN   
Universal Quality Index                           N/full                      NaN   
                                                  N/gray_b                    NaN   
                                                  N/gray_g                    NaN   
                                                  N/gray_r                    NaN   
                                                  P/full                   0.4231   

dataset                                                     orthologs_cytoglobin  \
algorithm                                         channel                          
Control with Clustal Omega                        N/full                     NaN   
                                                  N/gray_b                   NaN   
                                                  N/gray_g                   NaN   
                                                  N/gray_r                   NaN   
Greedy Sliced Structural Similarity Index Measure N/full                     NaN   
                                                  N/gray_r                   NaN   
                                                  P/full                     NaN   
Universal Quality Index                           N/full                  0.4231   
                                                  N/gray_b                0.4231   
                                                  N/gray_g                0.4231   
                                                  N/gray_r                0.4231   
                                                  P/full                  0.4231   

dataset                                                     orthologs_hemoglobin_beta  \
algorithm                                         channel                               
Control with Clustal Omega                        N/full                          NaN   
                                                  N/gray_b                     0.0000   
                                                  N/gray_g                     0.0000   
                                                  N/gray_r                        NaN   
Greedy Sliced Structural Similarity Index Measure N/full                   